## **Routing and Retrieving EV Charging Stations**

For routing this notebook uses the api of open routing service. This is a service based on open street map, but it requires authentication and works with credits. The education package gives us the following credits: (10.000 per day / 40 per minute) so it should be fine like that.

For more detailled information, have a look at:

- [ors directions API examples and playground](https://openrouteservice.org/dev/#/api-docs/v2/directions/{profile}/get)

Connect via API Key, choose a start and end address

In [8]:
import requests
import os

api_key = '5b3ce3597851110001cf624834ddb0fae46447969085ea7efe629c8f' #os.environ.get('ORS_API_KEY')
print(api_key)
start_address = "86 Gilman Street, Somerville, MA 02145"
end_address = "1 Harvard Yard, Cambridge, MA 02138"

converted_SA = start_address.replace(" ", "%20")
converted_EA = end_address.replace(" ", "%20")

country = "US"
resultSize = 1

5b3ce3597851110001cf624834ddb0fae46447969085ea7efe629c8f


Get coordinates for start and end address

In [9]:
def get_coordinates(api_key, converted_address, country, result_size=1):
    headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    }
    url = f'https://api.openrouteservice.org/geocode/search?api_key={api_key}&text={converted_address}&boundary.country={country}&size={result_size}'
    response = requests.get(url, headers=headers)
    data = response.json()

    if "features" in data and len(data["features"]) > 0:
        coordinates = data["features"][0]["geometry"]["coordinates"]
        latitude, longitude = coordinates
        return latitude, longitude
    else:
        return None, None
    
start_latitude, start_longitude = get_coordinates(api_key, converted_SA, country, resultSize)
end_latitude, end_longitude = get_coordinates(api_key, converted_EA, country, resultSize)
print(f"Start Latitude: {start_latitude}, Start Longitude: {start_longitude}")
print(f"End Latitude: {end_latitude}, End Longitude: {end_longitude}")

start_address = f"{start_latitude},{start_longitude}"
end_address = f"{end_latitude},{end_longitude}"

Start Latitude: -71.092508, Start Longitude: 42.385904
End Latitude: -71.115987, End Longitude: 42.38036


Get the route from the start to end address, extract duration

In [10]:
url = f"https://api.openrouteservice.org/v2/directions/driving-car?api_key={api_key}&start={start_address}&end={end_address}"

response = requests.get(url)
data = response.json()

if "features" in data and len(data["features"]) > 0:
    duration = data["features"][0]["properties"]["segments"][0]["duration"]
    distance = data["features"][0]["properties"]["segments"][0]["distance"]
    print(f"Duration of travel: {duration/60} minutes for a distance of {distance/1000} km")
else:
    print("Duration not found in the response.")


Duration of travel: 7.78 minutes for a distance of 3.5416 km


## Get EV charging stations within an area
For that we will use the osm overpass API
- [Documentation](https://wiki.openstreetmap.org/wiki/Overpass_API)
- [Test queries](https://overpass-turbo.eu)

In [11]:
import requests

# Define your area of interest
min_lat, min_lon, max_lat, max_lon = 42.35, -71.15, 42.40, -71.10
bbox = (min_lat, min_lon, max_lat, max_lon)

# Construct query
query = f'''
[out:json];
(
  node["amenity"="charging_station"]({bbox[0]}, {bbox[1]}, {bbox[2]}, {bbox[3]});
  way["amenity"="charging_station"]({bbox[0]}, {bbox[1]}, {bbox[2]}, {bbox[3]});
  relation["amenity"="charging_station"]({bbox[0]}, {bbox[1]}, {bbox[2]}, {bbox[3]});
);
out center;
'''

# Make the API request
overpass_url = "https://z.overpass-api.de/api/interpreter"
response = requests.post(overpass_url, data=query)

# visualize results
charging_stations = []

if response.status_code == 200:
    parsed_data = response.json()
    
    for element in parsed_data['elements']:
        if element.get('tags', {}).get('amenity') == 'charging_station':
            if element['type'] == 'node':
                lat = element.get('lat', None)
                lon = element.get('lon', None)
            elif element['type'] == 'way' and 'center' in element:
                lat = element['center'].get('lat', None)
                lon = element['center'].get('lon', None)
            else:
                lat = 'Unknown'
                lon = 'Unknown'
            name = element.get('tags', {}).get('name', 'Unknown Name')
            address = element.get('tags', {}).get('address', 'Unknown Address')
            charging_stations.append({
                'name': name,
                'address': address,
                'lat': lat,
                'lon': lon
            })
    print(charging_stations)
else:
    print(response.text)
    


[{'name': 'Unknown Name', 'address': 'Unknown Address', 'lat': 42.3892045, 'lon': -71.1327904}, {'name': 'ChargePoint', 'address': 'Unknown Address', 'lat': 42.3962469, 'lon': -71.1199394}, {'name': 'ChargePoint', 'address': 'Unknown Address', 'lat': 42.395375, 'lon': -71.1234511}, {'name': 'ChargePoint', 'address': 'Unknown Address', 'lat': 42.3659046, 'lon': -71.1055873}, {'name': 'Unknown Name', 'address': 'Unknown Address', 'lat': 42.369874, 'lon': -71.1031063}, {'name': 'Unknown Name', 'address': 'Unknown Address', 'lat': 42.3813403, 'lon': -71.1034341}]
